In [5]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

# 문제: zscore, cdf 계산
 성적데이터는 n이 적지만, (편의상)정규분포를 이룬다고 가정하자.

- 1) 성적데이터로 DataFrame을 생성.
- 2) zscore 컬럼을 생성. zscore를 계산하려면, 평균과 표준편차를 알아야 한다. 계산식에 F함수를 직접 사용하면 오류가 발생한다. 따로 평균과 표준편차를 구해서 계산식에서 사용해야 한다.
- 3) cdf 컬럼을 생성. scipy.stats.norm.cdf() 함수는 데이터타입을 float로 맞추어 주어야 한다. cdf는 평균=0, 표준편차=1을 기본 값으로 누적확률을 계산한다.

### 데이터

In [59]:
marks=[
    "김하나, English, 100",
    "김하나, Math, 80",
    "임하나, English, 70",
    "임하나, Math, 100",
    "김갑돌, English, 82.3",
    "김갑돌, Math, 98.5"
]

### DataFrame 생성
- 배열내에 문자열을 가지고 있는 경우, 바로 DataFrame으로 만들면 따옴표로 묶인 문자열이 한 값이 된다.
- RDD를 만들어 주고 map() 함수로 분리한 후 DataFrame을 만들어준다.

In [60]:
_marksRdd=spark.sparkContext.parallelize(marks).map(lambda x:x.split(','))

In [61]:
_marksDf=spark.createDataFrame(_marksRdd, schema=["name", "subject", "mark"])

In [62]:
_marksDf.printSchema()

root
 |-- name: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- mark: string (nullable = true)



스키마를 자동유추하면, mark 마저도 string으로 읽어온다. 소수점으로 변환이 필요하다.

### 데이터 타입 변경

In [63]:
from pyspark.sql.types import FloatType

# _marksDf = _marksDf.withColumn('markF_', F.expr("CAST(mark AS FLOAT)"))  # ok
_marksDf = _marksDf.withColumn('markF', _marksDf['mark'].cast(FloatType()))

In [64]:
_marksDf.printSchema()

root
 |-- name: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- mark: string (nullable = true)
 |-- markF: float (nullable = true)



In [65]:
_marksDf.show()

+------+--------+-----+-----+
|  name| subject| mark|markF|
+------+--------+-----+-----+
|김하나| English|  100|100.0|
|김하나|    Math|   80| 80.0|
|임하나| English|   70| 70.0|
|임하나|    Math|  100|100.0|
|김갑돌| English| 82.3| 82.3|
|김갑돌|    Math| 98.5| 98.5|
+------+--------+-----+-----+



### F.mean, F.stddev를 사용하면 zscore계산 오류
- `stats.zscore()` 함수는 배열을 입력받아서 zscore를 계산한다.
- 점수는 Column 데이터이라서, 배열로 변환해서 넘겨주려면 번거롭다.

zscore를 계산하려면, 평균과 표준편차를 알아야 한다.  
전체에 대한 평균을 계산하고, 이를 각 mark에 적용하는 계산식을 사용해 보자.  
이 때 `F.mean()`, `F.stddev()` 함수를 사용하면 오류가 발생한다.

In [66]:
from pyspark.sql import functions as F
zscoreUdf = F.udf(lambda x: (x-F.mean(x))/F.stddev) # does not work

In [67]:
zscoreUdf

<function __main__.<lambda>(x)>

### mean, stdev를 별도 계산해서 zscore 계산
- 평균과 표준편차를 먼저 구하고 계산식에 넣어주자.  
- **컬럼명은 아래와 같이 따옴표로 혹은 `F.col('markF')`로 넣어주어도 된다.**

In [68]:
from pyspark.sql import functions as F

_markStats = _marksDf.select(
    F.mean('markF').alias('mean'), # 평균 # alias: 이름 변경
    F.stddev('markF').alias('std') # 표준편차
).collect()

In [69]:
_markStats

[Row(mean=88.46666717529297, std=12.786190172956093)]

In [70]:
meanMark = _markStats[0]['mean']
stdMark = _markStats[0]['std']

In [71]:
meanMark

88.46666717529297

In [72]:
stdMark

12.786190172956093

또는 2차원 인덱스를 사용하여 평균을 읽어도 된다.

In [73]:
_markStats[0][0]

88.46666717529297

In [74]:
_markStats[0][1]

12.786190172956093

zscore를 계산하려면, FloatType으로 형변환 해준다.
- zscore: 표준편차기준 얼마나 떨어져있는가?

In [75]:
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType

zscoreUdf = F.udf(lambda x: (x-meanMark)/stdMark, FloatType()) # return as FloatType

In [76]:
#이렇게 해도 나오긴 한다. 그리고 zscore가 자릿수가 길~게나옴
# _marksDf=_marksDf.withColumn("zscore", (_marksDf['markF']-meanMark)/stdMark)

In [78]:
_marksDf=_marksDf.withColumn("zscore", zscoreUdf(_marksDf['markF']))

In [79]:
_marksDf.show()

+------+--------+-----+-----+-----------+
|  name| subject| mark|markF|     zscore|
+------+--------+-----+-----+-----------+
|김하나| English|  100|100.0|  0.9020148|
|김하나|    Math|   80| 80.0| -0.6621728|
|임하나| English|   70| 70.0| -1.4442666|
|임하나|    Math|  100|100.0|  0.9020148|
|김갑돌| English| 82.3| 82.3|-0.48229098|
|김갑돌|    Math| 98.5| 98.5| 0.78470075|
+------+--------+-----+-----+-----------+



### cdf 계산
norm.cdf는 numpy.float64를 반환하는데, spark에서 사용하지 않는 데이터타입이다. float()로 형변환을 해주자.

In [24]:
from scipy.stats import norm
type(norm.cdf(1)) #float64는 스파크가 이해할수없다

numpy.float64

In [80]:
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType

#bad_norm_cdf = F.udf(lambda x: norm.cdf(x), FloatType()) # FloatType() does not work
normCdf = F.udf(lambda x: float(norm.cdf(x))) # FloatType() does not work

In [84]:
normCdf 

<function __main__.<lambda>(x)>

#### 원점수를 사용하면 1.0¶
cdf는 평균=0, 표준편차=1을 기본 값으로 누적확률을 계산한다. 점수가 그런 범위에 있지 않고 훨씬 넘어가므로 1.0이 계산된다.

In [85]:
_marksDf.withColumn("cdf", normCdf(_marksDf['markF'])).show()

+------+--------+-----+-----+-----------+---+
|  name| subject| mark|markF|     zscore|cdf|
+------+--------+-----+-----+-----------+---+
|김하나| English|  100|100.0|  0.9020148|1.0|
|김하나|    Math|   80| 80.0| -0.6621728|1.0|
|임하나| English|   70| 70.0| -1.4442666|1.0|
|임하나|    Math|  100|100.0|  0.9020148|1.0|
|김갑돌| English| 82.3| 82.3|-0.48229098|1.0|
|김갑돌|    Math| 98.5| 98.5| 0.78470075|1.0|
+------+--------+-----+-----+-----------+---+



#### zscore를 사용하여 cdf 계산
zscore는 평균 0 표준편차 1을 잘 가지고있다

In [86]:
_marksDf=_marksDf.withColumn("cdf", normCdf(_marksDf['zscore']))

In [87]:
_marksDf.show()

+------+--------+-----+-----+-----------+-------------------+
|  name| subject| mark|markF|     zscore|                cdf|
+------+--------+-----+-----+-----------+-------------------+
|김하나| English|  100|100.0|  0.9020148| 0.8164754981807292|
|김하나|    Math|   80| 80.0| -0.6621728| 0.2539302463290559|
|임하나| English|   70| 70.0| -1.4442666| 0.0743320011235712|
|임하나|    Math|  100|100.0|  0.9020148| 0.8164754981807292|
|김갑돌| English| 82.3| 82.3|-0.48229098|0.31479962882028223|
|김갑돌|    Math| 98.5| 98.5| 0.78470075| 0.7836854740814176|
+------+--------+-----+-----+-----------+-------------------+



### Window 함수를 사용하여 zscore 계산
전체에 대한 평균점수를 컬럼으로 만드려면 Window 기능을 사용해야 한다.

#### 전체 Window
점수평균이라고 하면, Spark는 어떤 평균인지 모른다.
(사람별 점수평균인지, 과목별평균인지 알려주어야 한다.)  

정작 우리가 필요한 것은 전체점수의 평균이다. `rowsBetween(-sys.maxsize, sys.maxsize)`는 최대, 최소 값으로 윈도우를 정한다.

In [91]:
import sys
from pyspark.sql.window import Window
byAll = Window.rowsBetween(-sys.maxsize, sys.maxsize) #모든 행으로 window 만들겠다는 의미

#### 전체의 평균, 표준편차 컬럼을 만들고 계산
전체 Window에 대해 평균, 표준편차와 과목별 평균을 계산해보자.  
이 때 평균, 표준편차를 컬럼으로 만든 후에 zscore를 계산해보자.

In [92]:
from pyspark.sql import functions as F
_marksDf = _marksDf.withColumn("mean", F.avg(_marksDf['markF']).over(byAll)) #over 이용해서 전체 window 넘겨준다

In [93]:
_marksDf = _marksDf.withColumn("stddev", F.stddev(_marksDf['markF']).over(byAll))

과목별 평균은 직접 zscore 계산에 필요하지 않지만 덤으로 추가한다.

In [94]:
from pyspark.sql.window import Window

bySubject = Window.partitionBy('subject')
_marksDf = _marksDf.withColumn("meanBySubject", F.avg(_marksDf['markF']).over(bySubject))# 과목평균컬럼

In [95]:
_marksDf.show(_marksDf.count(), truncate=False) #_marksDf.count() 모든 행 보여준다 # truncate=False 값이 길어도 자르지 않고

+------+--------+-----+-----+-----------+-------------------+-----------------+------------------+-----------------+
|name  |subject |mark |markF|zscore     |cdf                |mean             |stddev            |meanBySubject    |
+------+--------+-----+-----+-----------+-------------------+-----------------+------------------+-----------------+
|김하나| English| 100 |100.0|0.9020148  |0.8164754981807292 |88.46666717529297|12.786190172956093|84.10000101725261|
|임하나| English| 70  |70.0 |-1.4442666 |0.0743320011235712 |88.46666717529297|12.786190172956093|84.10000101725261|
|김갑돌| English| 82.3|82.3 |-0.48229098|0.31479962882028223|88.46666717529297|12.786190172956093|84.10000101725261|
|김하나| Math   | 80  |80.0 |-0.6621728 |0.2539302463290559 |88.46666717529297|12.786190172956093|92.83333333333333|
|임하나| Math   | 100 |100.0|0.9020148  |0.8164754981807292 |88.46666717529297|12.786190172956093|92.83333333333333|
|김갑돌| Math   | 98.5|98.5 |0.78470075 |0.7836854740814176 |88.46666717529297|12.

In [88]:
#이름으로도 되나?
byName = Window.partitionBy('name')
name_marksDf = _marksDf.withColumn("meanByName", F.avg(_marksDf['markF']).over(byName))# 이름별 평균내기도 가능!

In [89]:
name_marksDf.show()

+------+--------+-----+-----+-----------+-------------------+----------------+
|  name| subject| mark|markF|     zscore|                cdf|      meanByName|
+------+--------+-----+-----+-----------+-------------------+----------------+
|김갑돌| English| 82.3| 82.3|-0.48229098|0.31479962882028223|90.4000015258789|
|김갑돌|    Math| 98.5| 98.5| 0.78470075| 0.7836854740814176|90.4000015258789|
|김하나| English|  100|100.0|  0.9020148| 0.8164754981807292|            90.0|
|김하나|    Math|   80| 80.0| -0.6621728| 0.2539302463290559|            90.0|
|임하나| English|   70| 70.0| -1.4442666| 0.0743320011235712|            85.0|
|임하나|    Math|  100|100.0|  0.9020148| 0.8164754981807292|            85.0|
+------+--------+-----+-----+-----------+-------------------+----------------+



In [35]:
_marksDf = _marksDf.withColumn("zscore1", (F.col('markF')-F.col('mean'))/F.col('stddev'))

In [36]:
_marksDf.select('zscore', 'zscore1').show(_marksDf.count())

+-----------+-------------------+
|     zscore|            zscore1|
+-----------+-------------------+
|  0.9020148|  0.902014804151829|
| -0.6621728| -0.662172786480269|
| -1.4442666| -1.444266581796318|
|  0.9020148|  0.902014804151829|
|-0.48229098|-0.4822909748814927|
| 0.78470075| 0.7847007348544217|
+-----------+-------------------+



#### 전체의 평균, 표준편차 컬럼을 만들지 않고 계산
또는 직접 Window 함수를 직접 사용하여 zscore를 계산할 수도 있다.

In [37]:
_marksDf = _marksDf.withColumn("zscore2", (F.col('markF')-F.avg('markF').over(byAll))/F.stddev('markF').over(byAll))

In [38]:
_marksDf.select('zscore', 'zscore1', 'zscore2').show()

+-----------+-------------------+-------------------+
|     zscore|            zscore1|            zscore2|
+-----------+-------------------+-------------------+
|  0.9020148|  0.902014804151829|  0.902014804151829|
| -0.6621728| -0.662172786480269| -0.662172786480269|
| -1.4442666| -1.444266581796318| -1.444266581796318|
|  0.9020148|  0.902014804151829|  0.902014804151829|
|-0.48229098|-0.4822909748814927|-0.4822909748814927|
| 0.78470075| 0.7847007348544217| 0.7847007348544217|
+-----------+-------------------+-------------------+

